# Screening Task - 1
#### Write a web scraper to collect google news (from here).A sample screenshot of the google news is shown above. Both, main news and sub news are to be collected from the whole page. This news is to be preprocessed, summarized, and stored in separate tables. Write a small function to search exact words like surge, acquisitions, and IPO (initial public offering).

## Import dependencies

In [1]:
import urllib.request
from bs4 import BeautifulSoup  
from newspaper import Article
import nltk
#un-comment as requried
#nltk.download('punkt')
#nltk.download('stopwords')

## Select Requried URL

In [2]:
#biz
url="https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtVnVHZ0pKVGlnQVAB?hl=en-IN&gl=IN&ceid=IN%3Aen"

#tech
#url='https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGRqTVhZU0FtVnVHZ0pKVGlnQVAB?hl=en-IN&gl=IN&ceid=IN%3Aen'

#ent
#url='https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNREpxYW5RU0FtVnVHZ0pKVGlnQVAB?hl=en-IN&gl=IN&ceid=IN%3Aen'

#health
#url='https://news.google.com/topics/CAAqIQgKIhtDQkFTRGdvSUwyMHZNR3QwTlRFU0FtVnVLQUFQAQ?hl=en-IN&gl=IN&ceid=IN%3Aen'

## Get HTML from the target page

In [3]:
page=urllib.request.Request(url,headers={'User-Agent': 'Mozilla/5.0'}) 
infile=urllib.request.urlopen(page).read()
soup = BeautifulSoup(infile, 'html.parser')

In [4]:
all_tabs=soup.find(attrs={"class":"fe4pJf"})
if all_tabs==None:
    print("Failed")

In [5]:
#extracting a list of elements containing the different new cards
tab=all_tabs(attrs={"class":'xrnccd F6Welf R7GTQ keNKEd j7vNaf'}) 
len(tab)

54

## Data Extraction And Storage

All the data will be stored in a dictionary format with the key being the news-title and will have a nested dictionary as value for each key.<br>
{	<br>
 <b>NEWS_TITLE_1</b>:<br>
 {<br>
     <b>SUB</b> : [ A list of sub-news topics under the news-title],<br>
     <b>LINKS</b> : [ A list of links which contained in the news article ],<br>
     <b>SUMMARY</b> : “String :  summary of the article from the original website”.<br>
 }<br>
    
 _ _ _<br><br>
 _ _ _<br><br>

 <b>NEWS_TITLE_N</b>:<br>
 {<br>
     <b>SUB</b> : [ A list of sub-news topics under the news-title],<br>
     <b>LINKS</b> : [ A list of links which contained in the news article ],<br>
     <b>SUMMARY</b> : “String :  summary of the article from the original website”.<br>
 }<br>
    
}

In [44]:
db={}
logs=[]#data collection and other exceptions and runtime errors will be logged here

In [45]:
for t in tab:
    title=BeautifulSoup.getText(t.find(attrs={"class":"DY5T1d"}))
    sub=list(map(BeautifulSoup.get_text,t.find_all(attrs={"class":"ipQwMb ekueJc gEATFF RD0gLb"})))
    link=list(set(['https://news.google.com/'+x['href'][2:] for x in t.find_all('a',href=True)]))
    summary=""
    try:
        article = Article(link[0], language="en") # en for English 
        #To download the article 
        article.download() 
        #To parse the article 
        article.parse() 
        #To perform natural language processing ie..nlp 
        article.nlp() 
        summary=article.summary
    except Exception as e:
        logs.append({"title":title,"error":e})
    finally:
        db[title]={"SUB":sub,"LINKS":link,"SUMMARY":summary}

In [46]:
#db

In [47]:
logs

[{'title': 'How Tesla And Other Automotive Manufacturers Are Fighting Coronavirus',
  'error': newspaper.article.ArticleException('Article `download()` failed with 403 Client Error: Forbidden for url: https://www.bizjournals.com/sanjose/news/2020/03/20/elon-musk-coronavirus-ventilators-tesla.html on URL https://news.google.com/articles/CBMiYGh0dHBzOi8vd3d3LmJpempvdXJuYWxzLmNvbS9zYW5qb3NlL25ld3MvMjAyMC8wMy8yMC9lbG9uLW11c2stY29yb25hdmlydXMtdmVudGlsYXRvcnMtdGVzbGEuaHRtbNIBZGh0dHBzOi8vd3d3LmJpempvdXJuYWxzLmNvbS9zYW5qb3NlL25ld3MvMjAyMC8wMy8yMC9lbG9uLW11c2stY29yb25hdmlydXMtdmVudGlsYXRvcnMtdGVzbGEuYW1wLmh0bWw?hl=en-IN&gl=IN&ceid=IN%3Aen')}]

### Converting the dict to a pandas dataframe

In [16]:
import pandas as pd

In [17]:
df=pd.DataFrame(data=db).transpose()

##### data can be stored in other formats , for now .json is being used

In [18]:
df.to_json('db.json')

In [34]:
df.head()

,SUB,LINKS,SUMMARY
Coronavirus: Maruti Suzuki halts production at Haryana plants,[Coronavirus: Maruti Suzuki halts production a...,[https://news.google.com/articles/CBMiigFodHRw...,"New Delhi: Maruti Suzuki India Ltd, country’s ..."
Coronavirus | Anand Mahindra offers to convert Mahindra resorts to temporary care facilities,[Coronavirus | Anand Mahindra offers to conver...,[https://news.google.com/publications/CAAqBwgK...,The YSRCP government in Andhra Pradesh has dec...
Coronavirus update: Calls for rate cut grow louder,[Coronavirus update: Calls for rate cut grow l...,[https://news.google.com/publications/CAAqBwgK...,Delhi Chief Minister Arvind Kejriwal on Sunday...
Hero MotoCorp Announces Global Shutdown Of Manufacturing Plants Amidst Coronavirus Pandemic,[Hero MotoCorp Announces Global Shutdown Of Ma...,[https://news.google.com/stories/CAAqbQgKImdDQ...,"World's largest two-wheeler maker, Hero MotoCo..."
"Centre nod to incentive scheme of Rs 40,995 cr for electronics firms","[Centre nod to incentive scheme of Rs 40,995 c...",[https://news.google.com/publications/CAAqBwgK...,Tata Motors on Friday said it has decided to r...


### Searching from keywords

##### Utility function to clean and speed up word searching by eliminating stop words

In [20]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
def filter_summary(text):
    stop_words = set(stopwords.words('english')) 

    word_tokens = word_tokenize(text) 

    filtered_sentence = [w for w in word_tokens if not w in stop_words] 

    filtered_sentence = [] 

    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    
    return filtered_sentence

###### Driver function to loop through the dataframe and call the above utility function 

In [21]:
def search_for_keyword(text):
    res=[]
    text=text.strip().lower()
    for i in df.index:
        search_space=' '.join([i,df.loc[i]['SUMMARY'],' '.join(df.loc[i]['SUB'])])
        if text in filter_summary(search_space.strip().lower()) :
            res.append(i)

    
    return df.loc[res]
    
    

In [31]:
search_for_keyword('acquisitions')

,SUB,LINKS,SUMMARY
"Faced with Over 75 lakh Job Cuts, US Airlines Warn Govt of 'Draconian' Steps in Absence of Help","[Faced with Over 75 lakh Job Cuts, US Airlines...",[https://news.google.com/publications/CAAqBwgK...,I have lived with the same man for more than a...


In [32]:
search_for_keyword('surge')

,SUB,LINKS,SUMMARY
"Up to 150% surge! After coronavirus hit, this stock could be a multibagger in the making!","[Up to 150% surge! After coronavirus hit, this...",[https://news.google.com/articles/CBMihwFodHRw...,Motilal Oswal has given a buy rating to Mother...


In [33]:
search_for_keyword('amazon')

,SUB,LINKS,SUMMARY
Amazon CEO Jeff Bezos warns workers that coronavirus will 'get worse before it gets better',[Amazon CEO Jeff Bezos warns workers that coro...,[https://news.google.com/articles/CBMitwFodHRw...,Amazon CEO Jeff Bezos said in a statement on S...
"COAI asks Netflix, Hotstar, other OTTs to lower streaming quality to reduce internet overload","[COAI asks Netflix, Hotstar, other OTTs to low...",[https://news.google.com/publications/CAAqBggK...,If there is ever a discussion about the greate...
Coronavirus: YouTube follows Netflix and cuts Europe streaming quality - Business Insider,[Coronavirus: YouTube follows Netflix and cuts...,[https://news.google.com/publications/CAAqBggK...,If there is ever a discussion about the greate...


## END